In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as pretrained_models

import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [3]:
import sys
sys.path.append("../common")

from Model_config import Models
from Net import Net
import LoaderUtil as util

## Set configuration

In [4]:
# batch_size should be less than 50 for VGG-16
batch_size = 32

## Load dataset

In [5]:
traindir = os.path.join('/home/data/ILSVRC2012/images', 'train')
valdir = os.path.join('/home/data/ILSVRC2012/images', 'val')

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transform_train = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])

transform_test = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize,
        ])

trainset = datasets.ImageFolder(root=traindir, transform=transform_train)
testset = datasets.ImageFolder(root=valdir, transform=transform_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

num_train_classes = len([name for name in os.listdir(traindir)])
num_valid_classes = len([name for name in os.listdir(valdir)])

print("num_train_classes = '{}'".format(num_train_classes))
print("num_valid_classes = '{}'".format(num_valid_classes))

num_train_classes = '1000'
num_valid_classes = '1000'


## (Resenet18) load params and test 

In [6]:
resnet = pretrained_models.resnet18(num_classes=1000, pretrained='imagenet')

print('pre-trained resnet18 is loaded')

pre-trained resnet18 is loaded


In [7]:
net = []

b = util.ConvBlockLoader(resnet.conv1, resnet.bn1, resnet.relu)

net.append(b)

net.append(resnet.maxpool)

for x in resnet.layer1:
    block = util.BasicBlockLoader(x)
    net.append(block)
    
for x in resnet.layer2:
    block = util.BasicBlockLoader(x)
    net.append(block)
    
for x in resnet.layer3:
    block = util.BasicBlockLoader(x)
    net.append(block)
    
for x in resnet.layer4:
    block = util.BasicBlockLoader(x)
    net.append(block)
    
net.append(resnet.avgpool)

net.append('Flatten')

b = util.FCBlockLoader(resnet.fc, option = 'FCOnly')
net.append(b)

In [8]:
model_gen = Models(dropout = False, batchnorm = True)
model_gen.ImagenetResnetConfig(num_layers = 18)

model = model_gen.GetImagenetResnet()
resnet18 = Net(model)

resnet18.LoadFromTorchvision(net)

torch.save(resnet18.GetStateDict(), './imagenet_resnet18_pretrained.pth')

print('Model saved')

Model saved


In [9]:
model_gen = Models(dropout = False, batchnorm = True)
model_gen.ImagenetResnetConfig(num_layers = 18)

model = model_gen.GetImagenetResnet()
resnet18 = Net(model)

state = torch.load('./imagenet_resnet18_pretrained.pth')
resnet18.LoadFromStateDict(state)

print('Reload model for the verification')

Reload model for the verification


In [10]:
correct = 0
correct_top5 = 0
correct_tmp = 0

total = 0
resnet18.TestMode()
resnet18.Gpu()

print('Inference for test set')

for i, data in enumerate(testloader, 0):
    images, labels = data

    outputs = resnet18(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
    _, predicted_top5 = torch.topk(outputs.data, 5)
    predicted_top5 = predicted_top5.t()
    predicted_top5_mat = predicted_top5.eq(labels.cuda().view(1,-1).expand_as(predicted_top5))

    for k in (1,5):
        correct_tmp = predicted_top5_mat[:k].view(-1).float().sum(0, keepdim=True)
    
    correct_top5 += correct_tmp[0]

    if i % 50 == 0:
        print ('.', end=' ')

print ('\n')
print('Top1 Acc: %4.2f %%, Top5 Acc: %4.2f %%' %(100 * correct / total, 100 * correct_top5 / total))

Inference for test set
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

Top1 Acc: 69.76 %, Top5 Acc: 89.08 %


## (Resenet34) load params and test

In [11]:
resnet = pretrained_models.resnet34(num_classes=1000, pretrained='imagenet')

print('pre-trained resnet34 is loaded')

pre-trained resnet34 is loaded


In [12]:
net = []

b = util.ConvBlockLoader(resnet.conv1, resnet.bn1, resnet.relu)

net.append(b)

net.append(resnet.maxpool)

for x in resnet.layer1:
    block = util.BasicBlockLoader(x)
    net.append(block)
    
for x in resnet.layer2:
    block = util.BasicBlockLoader(x)
    net.append(block)
    
for x in resnet.layer3:
    block = util.BasicBlockLoader(x)
    net.append(block)
    
for x in resnet.layer4:
    block = util.BasicBlockLoader(x)
    net.append(block)
    
net.append(resnet.avgpool)

net.append('Flatten')

b = util.FCBlockLoader(resnet.fc, option = 'FCOnly')
net.append(b)

In [13]:
model_gen = Models(dropout = False, batchnorm = True)
model_gen.ImagenetResnetConfig(num_layers = 34)

model = model_gen.GetImagenetResnet()
resnet34 = Net(model)

resnet34.LoadFromTorchvision(net)

torch.save(resnet34.GetStateDict(), './imagenet_resnet34_pretrained.pth')

print('Model saved')

Model saved


In [14]:
model_gen = Models(dropout = False, batchnorm = True)
model_gen.ImagenetResnetConfig(num_layers = 34)

model = model_gen.GetImagenetResnet()
resnet34 = Net(model)

state = torch.load('./imagenet_resnet34_pretrained.pth')
resnet34.LoadFromStateDict(state)

print('Reload model for the verification')

Reload model for the verification


In [15]:
correct = 0
correct_top5 = 0
correct_tmp = 0

total = 0

resnet34.TestMode()
resnet34.Gpu()

print('Inference for test set')

for i, data in enumerate(testloader, 0):
    images, labels = data

    outputs = resnet34(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
    _, predicted_top5 = torch.topk(outputs.data, 5)
    predicted_top5 = predicted_top5.t()
    predicted_top5_mat = predicted_top5.eq(labels.cuda().view(1,-1).expand_as(predicted_top5))

    for k in (1,5):
        correct_tmp = predicted_top5_mat[:k].view(-1).float().sum(0, keepdim=True)
    
    correct_top5 += correct_tmp[0]

    if i % 50 == 0:
        print ('.', end=' ')

print ('\n')
print('Top1 Acc: %4.2f %%, Top5 Acc: %4.2f %%' %(100 * correct / total, 100 * correct_top5 / total))

Inference for test set
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

Top1 Acc: 73.31 %, Top5 Acc: 91.42 %


## (VGG16_BN) load params and test

In [16]:
vggnet = pretrained_models.vgg16_bn(num_classes=1000, pretrained='imagenet')

print('pre-trained model is loaded')

pre-trained model is loaded


In [17]:
net = []

for x in vggnet.features:
    
    if isinstance(x, nn.Conv2d):
        b = []
        b.append(x)
        b.append('ConvBlock')
        net.append(b)
        
    elif isinstance(x, nn.BatchNorm2d) :
        net[-1].insert(-1, x)
        
    elif isinstance(x, nn.ReLU) :
        net[-1].insert(-1, x)
        
    elif isinstance(x, nn.Dropout2d) :
        net[-1].insert(-1, x)
        
    else :
        net.append(x)
    
net.append('Flatten')

for x in vggnet.classifier:
    if isinstance(x, nn.Linear):
        b = []
        b.append(x)
        b.append('FCBlock')
        net.append(b)
        
    elif isinstance(x, nn.BatchNorm1d) :
        net[-1].insert(-1, x)
        
    elif isinstance(x, nn.ReLU) :
        net[-1].insert(-1, x)
        
    elif isinstance(x, nn.Dropout) :
        net[-1].insert(-1, x)
        
    else :
        net.append(x)

In [18]:
model_gen = Models(dropout = False, batchnorm = True)
model_gen.ImagenetVgg16Config(dropout = False, batchnorm =True)

model = model_gen.GetImagenetVgg16()
vgg16_ = Net(model)

vgg16_.LoadFromTorchvision(net)

torch.save(vgg16_.GetStateDict(), './imagenet_vgg16_pretrained.pth')

print('Model saved')

Model saved


In [19]:
model_gen = Models(dropout = False, batchnorm = True)
model_gen.ImagenetVgg16Config(dropout = False, batchnorm =True)

model = model_gen.GetImagenetVgg16()
vgg16_ = Net(model)

state = torch.load('./imagenet_vgg16_pretrained.pth')
vgg16_.LoadFromStateDict(state)

print('Reload model for the verification')

Reload model for the verification


In [20]:
correct = 0
correct_top5 = 0
correct_tmp = 0

total = 0

vgg16_.TestMode()
vgg16_.Gpu()

print('Inference for test set')

for i, data in enumerate(testloader, 0):
    images, labels = data

    outputs = vgg16_(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
    _, predicted_top5 = torch.topk(outputs.data, 5)
    predicted_top5 = predicted_top5.t()
    predicted_top5_mat = predicted_top5.eq(labels.cuda().view(1,-1).expand_as(predicted_top5))

    for k in (1,5):
        correct_tmp = predicted_top5_mat[:k].view(-1).float().sum(0, keepdim=True)
    
    correct_top5 += correct_tmp[0]

    if i % 50 == 0:
        print ('.', end=' ')

print ('\n')
print('Top1 Acc: %4.2f %%, Top5 Acc: %4.2f %%' %(100 * correct / total, 100 * correct_top5 / total))

Inference for test set
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

Top1 Acc: 73.36 %, Top5 Acc: 91.52 %


## ResNet-50 load

In [21]:
resnet = pretrained_models.resnet50(num_classes=1000, pretrained='imagenet')

print('pre-trained resnet50 is loaded')

pre-trained resnet50 is loaded


In [22]:
net = []

b = util.ConvBlockLoader(resnet.conv1, resnet.bn1, resnet.relu)

net.append(b)

net.append(resnet.maxpool)

for x in resnet.layer1:
    block = util.BottleneckLoader(x)
    net.append(block)
    
for x in resnet.layer2:
    block = util.BottleneckLoader(x)
    net.append(block)
    
for x in resnet.layer3:
    block = util.BottleneckLoader(x)
    net.append(block)
    
for x in resnet.layer4:
    block = util.BottleneckLoader(x)
    net.append(block)
    
net.append(resnet.avgpool)

net.append('Flatten')

b = util.FCBlockLoader(resnet.fc, option = 'FCOnly')
net.append(b)

In [23]:
model_gen = Models(dropout = False, batchnorm = True)
model_gen.ImagenetResnetConfig(num_layers = 50, block_type = 'Bottleneck')

model = model_gen.GetImagenetResnet()
resnet50 = Net(model)

resnet50.LoadFromTorchvision(net)

torch.save(resnet50.GetStateDict(), './imagenet_resnet50_pretrained.pth')

print('Model saved')

Model saved


In [24]:
model_gen = Models(dropout = False, batchnorm = True)
model_gen.ImagenetResnetConfig(num_layers = 50, block_type = 'Bottleneck')

model = model_gen.GetImagenetResnet()
resnet50 = Net(model)

state = torch.load('./imagenet_resnet50_pretrained.pth')
resnet50.LoadFromStateDict(state)

print('Reload model for the verification')

Reload model for the verification


In [25]:
correct = 0
correct_top5 = 0
correct_tmp = 0

total = 0

resnet50.TestMode()
resnet50.Gpu()

print('Inference for test set')

for i, data in enumerate(testloader, 0):
    images, labels = data

    outputs = resnet50(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
    _, predicted_top5 = torch.topk(outputs.data, 5)
    predicted_top5 = predicted_top5.t()
    predicted_top5_mat = predicted_top5.eq(labels.cuda().view(1,-1).expand_as(predicted_top5))

    for k in (1,5):
        correct_tmp = predicted_top5_mat[:k].view(-1).float().sum(0, keepdim=True)
    
    correct_top5 += correct_tmp[0]

    if i % 50 == 0:
        print ('.', end=' ')

print ('\n')
print('Top1 Acc: %4.2f %%, Top5 Acc: %4.2f %%' %(100 * correct / total, 100 * correct_top5 / total))

Inference for test set
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

Top1 Acc: 76.13 %, Top5 Acc: 92.86 %


## DenseNet 121

In [26]:
densenet = pretrained_models.densenet121(num_classes=1000, pretrained='imagenet')

print('pre-trained densenet121 is loaded')

pre-trained densenet121 is loaded


In [27]:
net = []

features = densenet.features

b = []
b = util.ConvBlockLoader(features.conv0, features.norm0, features.relu0)

net.append(b)

net.append(features.pool0)

b = util.DenseBlockLoader(features.denseblock1, features.transition1.norm, features.transition1.relu)
net.append(b)

b = util.TransitionLoader(features.transition1)
net.append(b)

net.append(features.transition1.pool)

b = util.DenseBlockLoader(features.denseblock2, features.transition2.norm, features.transition2.relu)
net.append(b)

b = util.TransitionLoader(features.transition2)
net.append(b)

net.append(features.transition2.pool)

b = util.DenseBlockLoader(features.denseblock3, features.transition3.norm, features.transition3.relu)
net.append(b)

b = util.TransitionLoader(features.transition3)
net.append(b)

net.append(features.transition2.pool)

b = util.DenseBlockLoader(features.denseblock4, features.norm5)
net.append(b)

net.append(nn.AvgPool2d(kernel_size=7, stride=7, padding=0, ceil_mode=False, count_include_pad=True))

net.append('Flatten')

b = util.FCBlockLoader(densenet.classifier, option = 'FCOnly')
net.append(b)

In [28]:
model_gen = Models(dropout = False, batchnorm = True)
model_gen.ImagenetDensenetConfig(num_layers = 121)

model = model_gen.GetImagenetDensenet()
densenet121 = Net(model)

densenet121.LoadFromTorchvision(net)

torch.save(densenet121.GetStateDict(), './imagenet_densenet121_pretrained.pth')

print('Model saved')

Model saved


In [29]:
model_gen = Models(dropout = False, batchnorm = True)
model_gen.ImagenetDensenetConfig(num_layers = 121)

model = model_gen.GetImagenetDensenet()
densenet121 = Net(model)

state = torch.load('./imagenet_densenet121_pretrained.pth')
densenet121.LoadFromStateDict(state)

print('Reload model for the verification')

Reload model for the verification


In [30]:
correct = 0
correct_top5 = 0
correct_tmp = 0

total = 0

densenet121.TestMode()
densenet121.Gpu()

print('Inference for test set')

for i, data in enumerate(testloader, 0):
    images, labels = data

    outputs = densenet121(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
    _, predicted_top5 = torch.topk(outputs.data, 5)
    predicted_top5 = predicted_top5.t()
    predicted_top5_mat = predicted_top5.eq(labels.cuda().view(1,-1).expand_as(predicted_top5))

    for k in (1,5):
        correct_tmp = predicted_top5_mat[:k].view(-1).float().sum(0, keepdim=True)
    
    correct_top5 += correct_tmp[0]

    if i % 50 == 0:
        print ('.', end=' ')

print ('\n')
print('Top1 Acc: %4.2f %%, Top5 Acc: %4.2f %%' %(100 * correct / total, 100 * correct_top5 / total))

Inference for test set
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

Top1 Acc: 74.43 %, Top5 Acc: 91.97 %
